# Introduction

In this Jupyter notebook, I will be covering how I have used the amazing [XLSXWriter](https://xlsxwriter.readthedocs.io/).

While there is also [openpyxl](https://openpyxl.readthedocs.io/en/stable/), I really liked XLSXWriter much better.

While there are MANY ways to use this, I am going to cover how I use it to:
- read in an excel file
- create a new excel file
- some formatting
- centering columns

Be sure to [read the docs](https://xlsxwriter.readthedocs.io/) for additional things not covered here.

# Setup

You will need the following to be installed:
- python 3
- pandas
- xlsxwriter

`pip3 install pandas`

`pip3 install XlsxWriter`

If you don't already have a [virtual environment](https://github.com/ProsperousHeart/cheatsheets/blob/master/Tools/VirtualEnvironments.md) created, please do so now. You can even [use my cheat sheet](https://github.com/ProsperousHeart/cheatsheets/blob/master/Processes/virtualenvs.md). Inside this virtual environment is where you should install **pandas** and **xlsxwriter** ... The reason for this is not every project can use some of the later versions of packages, so you want to ensure you don't cause issues later down the line.

In this walkthrough, I also provide an XLSX file that will allow us to populate data. This comes from the [UCI Machine Learning Repository](http://archive.ics.uci.edu/ml/datasets/Air+Quality) and covers air quality from March 2004 to February 2005. While we will not necessarily be using this data for any other purpose other than how to use XLSXWriter, this is what the data entails:
- 9358 instances of hourly averaged responses from an array of 5 metal oxide chemical sensors embedded in an air quality chemical multisensor device
- device was located in a significantly polluted area at road level in an Italian city
- Ground Truth hourly averaged concentrations for:
    1. CO2
    2. SNO - tin oxide
    3. NMHC - Non Metanic Hydrocarbons in micrograms
    4. C6H6 - Benzene (micrograms)
    5. TIO2 - titanium dioxide
    6. NOX - Total Nitrogen Oxides (ppb)
    7. WOX - Tungsten oxide
    8. NO2 - Nitrogen Dioxide (microgram)
- includes date & time

# Code

The first step you need is data. In whatever format you get it, plug it into a pandas DataFrame. Whether from MongoDB, hard coded, or even another XLSX file it really doesn't matter.

For this example, we'll use the XLSX file mentioned above.

In [5]:
# add your imports
import pandas
import xlsxwriter # yes, install was XlsxWriter but import all lower case

FILE = 'files/AirQualityUCI.csv'

In [7]:
def read_file(filename_str:str):
    """
    This function takes in the path of a CSV file 
    and returns a DataFrame of the data.
    
    """
    
    pass

In [8]:
df = read_file(FILE)